In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from scipy.io import wavfile
import numpy as np

In [3]:
import sys
if "../src" not in sys.path:
    sys.path.append("../src")
from modist.modist_pcm_dataset import ModistPcmDataset, get_modist_pcm_collate_fn
from modist.mp3_info import get_lang_encoder

In [4]:
import json
with open("../data/mp3_split.json", "r") as fin:
    splits = json.load(fin)
train_mp3s = [x[0] for x in splits["train"]]
train_mp3s = [x.replace(".mp3", ".pcm") for x in train_mp3s]
pcm_list = ["../data/pcm/" + x for x in train_mp3s]

In [5]:
ds = ModistPcmDataset(pcm_list, speech_only=True)

In [6]:
len(ds.inventory)

13579

In [7]:
ds[0]

{'pcm_lang': 'Pinuyumayan',
 'category': 'speech',
 'ss': 20,
 'ee': 25,
 'samples': memmap([  552,  1685,  2717, ...,   205, -1594,  1175], dtype=int16)}

In [8]:
def dummy_feat_extractor(x, *args, **kwargs):
    data = [xx.shape for xx in x]    
    return type('DummyFeats', (object,), {"input_values": data})

lang_encoder = get_lang_encoder()

In [9]:
from transformers import Wav2Vec2FeatureExtractor
feat_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-base-960h")

In [10]:
collate_fn = get_modist_pcm_collate_fn(feat_extractor, lang_encoder, ds.sample_rate)

In [11]:
from torch.utils.data import DataLoader

In [12]:
train_loader = DataLoader(ds, batch_size=16, collate_fn=collate_fn, drop_last=True, shuffle=True)

In [13]:
from tqdm.auto import tqdm

In [14]:
## estimate batch count (all categories)
len(pcm_list) * 20 * 60 / 5 / 16

945.0

In [15]:
%%time 
for x in tqdm(train_loader):
    pass

  0%|          | 0/848 [00:00<?, ?it/s]

CPU times: user 37.4 s, sys: 5.48 s, total: 42.9 s
Wall time: 42.8 s


In [24]:
import psutil
mem = psutil.Process().memory_info()
print("RSS(GB): ", mem.rss/1024**3)
print("Virtual(GB): ", mem.vms/1024**3)

RSS(GB):  2.3067283630371094
Virtual(GB):  6.222900390625
